In [ ]:
# !pip install llama-parse
# !pip install llama-index-vector-stores-chroma
# !pip install llama-index-embeddings-huggingface
# !pip install llama-index-llms-langchain
# !pip install llama-index-llms-gemini

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()

In [ ]:
from llama_parse import LlamaParse  # pip install llama-parse
import nest_asyncio
nest_asyncio.apply()

parser = LlamaParse(
    api_key="llx-njIVCaPDqYOUfzRcFanrVBEYX6tuB8MXzBnCCGTZcxNs0HhY",  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown"  # "markdown" and "text" are available
)

# sync
documents = parser.load_data("386753.pdf")

In [ ]:
documents

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("test2")

# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

In [ ]:
from llama_index.core import Settings
import os
from llama_index.llms.gemini import Gemini

GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = Gemini(api_key=GEMINI_API_KEY)
Settings.llm = llm

In [ ]:
from llama_index.core import StorageContext
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

# Query Data
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("Economics Million USDrevenue at 2065?")
display((f"<b>{response}</b>"))

In [ ]:
import pprint
pprint(response)